In [1]:
import numpy as np
import pandas as pd
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
raw_data = pd.read_csv("data/Main_Data_SOM.csv")

In [4]:
columns = raw_data.columns.to_list()
print(columns)

['Index', 'd__Archaea;__;__;__;__;__', 'd__Archaea;p__Euryarchaeota;c__Methanobacteria;o__Methanobacteriales;f__Methanobacteriaceae;g__Methanobrevibacter', 'd__Archaea;p__Euryarchaeota;c__Methanobacteria;o__Methanobacteriales;f__Methanobacteriaceae;g__Methanosphaera', 'd__Archaea;p__Halobacterota;c__Methanomicrobia;o__Methanomicrobiales;f__Methanocorpusculaceae;g__Methanocorpusculum', 'd__Archaea;p__Halobacterota;c__Methanomicrobia;o__Methanomicrobiales;f__Methanomicrobiaceae;g__Methanoculleus', 'd__Archaea;p__Halobacterota;c__Methanomicrobia;o__Methanomicrobiales;f__Methanomicrobiaceae;g__Methanomicrobium', 'd__Archaea;p__Halobacterota;c__Methanomicrobia;o__Methanomicrobiales;f__Methanomicrobiaceae;g__uncultured', 'd__Archaea;p__Halobacterota;c__Methanosarcinia;o__Methanosarciniales;f__Methanosarcinaceae;g__Methanimicrococcus', 'd__Archaea;p__Halobacterota;c__Methanosarcinia;o__Methanosarciniales;f__Methanosarcinaceae;g__Methanosarcina', 'd__Archaea;p__Thermoplasmatota;c__Thermoplasma

In [5]:
# Ideally we want to have the genus of the microbes 
# But for now, drop columns where they couldn't identify genus or family
# (And keep ones where the family was identified but not the genus)
unidentified_microbes = [i for i in columns if i[-5:] == "__;__"]
print(f"Found {len(unidentified_microbes)} unidentified microbes:")
print(unidentified_microbes)

raw_data = raw_data.drop(unidentified_microbes, axis=1)

Found 33 unidentified microbes:
['d__Archaea;__;__;__;__;__', 'd__Bacteria;__;__;__;__;__', 'd__Bacteria;p__Actinobacteriota;c__Actinobacteria;__;__;__', 'd__Bacteria;p__Actinobacteriota;c__Actinobacteria;o__Corynebacteriales;__;__', 'd__Bacteria;p__Actinobacteriota;c__Coriobacteriia;__;__;__', 'd__Bacteria;p__Actinobacteriota;c__Coriobacteriia;o__Coriobacteriales;__;__', 'd__Bacteria;p__Bacteroidota;c__Bacteroidia;__;__;__', 'd__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;__;__', 'd__Bacteria;p__Firmicutes;__;__;__;__', 'd__Bacteria;p__Firmicutes;c__Bacilli;__;__;__', 'd__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;__;__', 'd__Bacteria;p__Firmicutes;c__Bacilli;o__Erysipelotrichales;__;__', 'd__Bacteria;p__Firmicutes;c__Bacilli;o__Lactobacillales;__;__', 'd__Bacteria;p__Firmicutes;c__Clostridia;__;__;__', 'd__Bacteria;p__Firmicutes;c__Clostridia;o__Clostridiales;__;__', 'd__Bacteria;p__Firmicutes;c__Clostridia;o__Eubacteriales;__;__', 'd__Bacteria;p__Firmicutes;c__Clos

In [6]:
# drop rows that begin with "Zero (Phase" for now
# TODO: check what they are
raw_data = raw_data.drop(labels=[0,41])

In [7]:
# Create a new column that separates out the day number in the experiment
def days(x):
    # this finds the integers and returns a list of integers found
    # we return the first one found
    # so for example in "2R6" we return 2
    return re.findall(r'\d+', x)[0]

raw_data['day'] = raw_data['Index'].map(days)

# Create a new column that separates out the experiment pH
def ph(x):
    if x[-1] in ["R", "M", "N"]:
        return 7.5
    else:
        # if we don't delete rows that say "Zero (Phase X)" etc then
        # this doesn't work
        return 6.0

raw_data["pH"] = raw_data['Index'].map(ph)

# Create a new column that separates out the temperature
def temperature(x):
    if x[-1] in ["R", "6"]:
        return "Control"
    elif x[-1] in ["M", "J"]:
        return "100C"
    elif x[-1] in ["N", "K"]:
        return "140C"
    else:
        # we shouldn't get here
        return "Uncategorized"

raw_data["temperature"] = raw_data['Index'].map(temperature)

In [8]:
# Now we have these columns separated out in case we want to group on them
# For analysis
raw_data[['Index', 'day', 'pH', 'temperature']].head(10)

,Index,day,pH,temperature
1,3R,3,7.5,Control
2,5R,5,7.5,Control
3,8R,8,7.5,Control
4,10R,10,7.5,Control
5,12R,12,7.5,Control
6,15R,15,7.5,Control
7,17R,17,7.5,Control
8,19R,19,7.5,Control
9,22R,22,7.5,Control
10,24R,24,7.5,Control


In [9]:
# make a list of the microbes

columns = raw_data.columns.to_list()
microbe_columns = [i for i in raw_data.columns.to_list() if ";" in i]
print(len(microbe_columns))  

494


In [10]:
raw_data.transpose()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86
Index,3R,5R,8R,10R,12R,15R,17R,19R,22R,24R,27R,29R,31R,33R,36R,3M,5M,8M,10M,12M,15M,17M,19M,22M,24M,3N,5N,8N,10N,12N,15N,17N,19N,22N,24N,27N,29N,31N,33N,36N,2R6,4R6,6R6,9R6,11R6,14R6,16R6,18R6,21R6,23R6,25R6,28R6,30R6,32R6,35R6,2J,4J,6J,9J,11J,14J,16J,18J,21J,23J,25J,28J,30J,32J,35J,2K,4K,6K,9K,11K,14K,16K,18K,21K,23K,25K,28K,30K,32K,35K
d__Archaea;p__Euryarchaeota;c__Methanobacteria;o__Methanobacteriales;f__Methanobacteriaceae;g__Methanobrevibacter,30,0,0,0,30,0,0,0,0,18,0,7,0,0,0,0,53,15,8,46,16,13,15,0,14,0,0,15,51,84,0,32,5,0,0,0,0,0,0,0,0,0,31,0,0,0,0,0,0,0,0,0,15,15,39,12,0,0,16,0,0,0,0,31,7,0,16,16,0,0,0,0,16,0,0,8,0,0,4,0,0,8,5,0,0
d__Archaea;p__Euryarchaeota;c__Methanobacteria;o__Methanobacteriales;f__Methanobacteriaceae;g__Methanosphaera,7,0,0,0,0,0,0,0,2,0,0,0,0,0,0,3,0,0,0,0,0,2,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0
d__Archaea;p__Halobacterota;c__Methanomicrobia;o__Methanomicrobiales;f__Methanocorpusculaceae;g__Methanocorpusculum,24,54,182,385,256,447,77,357,379,356,496,358,370,624,727,0,0,21,63,39,4,0,41,35,69,0,0,13,0,0,0,138,0,0,0,0,0,0,0,0,0,0,0,0,0,13,0,26,23,35,0,0,36,37,94,0,0,0,0,0,0,0,4,0,6,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,3,0
d__Archaea;p__Halobacterota;c__Methanomicrobia;o__Methanomicrobiales;f__Methanomicrobiaceae;g__Methanoculleus,0,0,0,0,0,0,82,18,0,21,22,29,39,105,214,0,0,31,83,69,148,348,195,113,185,0,2,0,0,83,318,50,372,397,268,286,560,423,242,440,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Hexanoic Acid,0.0,15.527975,8.366359,20.882474,7.763987,19.20979,0.0,0.0,0.0,11.043242,17.421774,0.0,43.437633,0.0,0.0,0.0,6.224266,4.216604,4.216604,5.875331,12.716661,0.0,32.060148,27.576149,19.945859,0.0,11.980591,0.0,13.653276,15.795369,18.339289,0.0,12.382418,35.741228,7.094767,4.41715,45.981552,31.792019,26.103276,0.0,58.129633,83.261768,153.80761,135.333889,115.255067,171.276398,160.131414,216.789275,205.879731,245.435737,263.708177,245.30204,193.296033,173.819583,144.704446,0.0,17.201394,14.256382,58.765062,61.911353,69.072969,137.141005,151.665516,218.730823,220.169537,188.54464,131.251717,242.75812,54.012934,194.032836,0.0,8.433208,79.112747,78.777769,32.929914,12.449266,75.798599,151.46497,408.613169,763.079707,358.950536,130.18067,293.358102,204.473706,629.418503
Methane,7.989,11.704,13.839,15.246,16.736,18.152,19.579,19.49,23.049,26.339,36.113,42.048,52.89,63.905,58.352,0.529,6.572,10.992,14.061,19.45,35.054,39.115,45.993,49.739,57.433,0.131,1.136,0.0,4.954,7.517,11.38,9.66,12.456,12.702,12.575,18.102,17.445,23.144,14.91,14.671,2.624,4.785,7.621,11.193,10.947,11.287,12.072,11.549,13.033,12.562,12.995,14.219,14.471,14.882,13.014,0.0,0.326,1.023,2.414,3.314,5.62,7.154,5.082,5.815,4.599,4.047,8.85,4.292,4.842,9.883,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.445,0.0,0.0,0.0,0.0,0.0
day,3,5,8,10,12,15,17,19,22,24,27,29,31,33,36,3,5,8,10,12,15,17,19,22,24,3,5,8,10,12,15,17,19,22,24,27,29,31,33,36,2,4,6,9,11,14,16,18,21,23,25,28,30,32,35,2,4,6,9,11,14,16,18,21,23,25,28,30,32,35,2,4,6,9,11,14,16,18,21,23,25,28,30,32,35
pH,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6

# Feature Selection

## Random Forest
[[https://www.geeksforgeeks.org/machine-learning/feature-selection-using-random-forest/]]

### Random Forest Classifier

In [11]:
# first create our classes
raw_data["Hexanoic Acid"].quantile([0,.33,.66,1])

0.00      0.000000
0.33     12.641790
0.66     97.338819
1.00    763.079707
Name: Hexanoic Acid, dtype: float64

In [12]:
def classify_hexanoic(x):
    """Classify hexanoic acid levels according (roughly)
    to 33% and 66% quantiles. So this is on a log scale.
    0 = levels between 0 - 10 (mg/L iirc)
    1 = levels between 10 - 100
    2 = levels between 10 - and max=763. 
    """
    if x <= 10:
        return 0
    elif x<= 100:
        return 1
    else:
        return 2
raw_data["hexanoic_class"] = raw_data["Hexanoic Acid"].map(classify_hexanoic)

In [13]:
n_estimators = 500
random_seed = 11
X = raw_data[microbe_columns]
y = raw_data["hexanoic_class"]
rf = RandomForestClassifier(n_estimators=n_estimators, random_state=random_seed)
rf.fit(X, y)

importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': microbe_columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
top_features = feature_importance_df['Feature'][:10].values

print([i.split(';')[-1] for i in top_features])
# print(feature_importance_df)

['g__Incertae_Sedis', 'g__Caproiciproducens', 'g__Asteroleplasma', '__', 'g__NK4A214_group', 'g__Oligella', 'g__Proteiniphilum', 'g__Fonticella', '__', 'g__Phascolarctobacterium']


In [14]:
n_estimators = 500
random_seed = 118
X = raw_data[microbe_columns]
y = raw_data["hexanoic_class"]
rf = RandomForestClassifier(n_estimators=n_estimators, random_state=random_seed)
rf.fit(X, y)

importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': microbe_columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
top_features = feature_importance_df['Feature'][:10].values

print([i.split(';')[-1] for i in top_features])
# print(feature_importance_df)

['g__Incertae_Sedis', 'g__Caproiciproducens', 'g__Fonticella', 'g__Oligella', 'g__Asteroleplasma', 'g__Proteiniphilum', 'g__Oscillibacter', 'g__NK4A214_group', '__', 'g__Prevotellaceae_YAB2003_group']


In [15]:
n_estimators = 500
random_seed = 1700
X = raw_data[microbe_columns]
y = raw_data["hexanoic_class"]
rf = RandomForestClassifier(n_estimators=n_estimators, random_state=random_seed)
rf.fit(X, y)

importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': microbe_columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
top_features = feature_importance_df['Feature'][:10].values

print([i.split(';')[-1] for i in top_features])
# print(feature_importance_df)

['g__Incertae_Sedis', 'g__Caproiciproducens', 'g__NK4A214_group', 'g__Fonticella', '__', 'g__Oligella', 'g__Prevotella', 'g__Proteiniphilum', 'g__Ruminiclostridium', 'g__Acholeplasma']


### Random Forest Regressor

In [16]:
n_estimators = 500
random_seed = 7
X = raw_data[microbe_columns]
y = raw_data["Hexanoic Acid"]
rf = RandomForestRegressor(n_estimators=n_estimators, random_state=random_seed)
rf.fit(X, y)

importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': microbe_columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
top_features = feature_importance_df['Feature'][:10].values

print([i.split(';')[-1] for i in top_features])
print(feature_importance_df)

['g__Incertae_Sedis', 'g__Mailhella', 'g__Peptostreptococcales-Tissierellales', 'g__Caproiciproducens', 'g__Treponema', 'g__Phascolarctobacterium', 'g__Rikenellaceae_RC9_gut_group', 'g__Izemoplasmatales', 'g__Gastranaerophilales', 'g__Asteroleplasma']
                                               Feature    Importance
284  d__Bacteria;p__Firmicutes;c__Clostridia;o__Osc...  3.130353e-01
138  d__Bacteria;p__Desulfobacterota;c__Desulfovibr...  1.346951e-01
345  d__Bacteria;p__Firmicutes;c__Clostridia;o__Pep...  1.207896e-01
306  d__Bacteria;p__Firmicutes;c__Clostridia;o__Osc...  8.298070e-02
467  d__Bacteria;p__Spirochaetota;c__Spirochaetia;o...  5.107282e-02
370  d__Bacteria;p__Firmicutes;c__Negativicutes;o__...  2.634994e-02
95   d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...  2.244162e-02
174  d__Bacteria;p__Firmicutes;c__Bacilli;o__Izemop...  2.125158e-02
133  d__Bacteria;p__Cyanobacteria;c__Vampirivibrion...  1.108018e-02
155  d__Bacteria;p__Firmicutes;c__Bacilli;o__Erysip...  1.

In [17]:
n_estimators = 500
random_seed = 89
X = raw_data[microbe_columns]
y = raw_data["Hexanoic Acid"]
rf = RandomForestRegressor(n_estimators=n_estimators, random_state=random_seed)
rf.fit(X, y)

importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': microbe_columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
top_features = feature_importance_df['Feature'][:10].values

print([i.split(';')[-1] for i in top_features])
print(feature_importance_df)

['g__Incertae_Sedis', 'g__Mailhella', 'g__Peptostreptococcales-Tissierellales', 'g__Caproiciproducens', 'g__Treponema', 'g__Izemoplasmatales', 'g__Phascolarctobacterium', 'g__Rikenellaceae_RC9_gut_group', 'g__Anaerosalibacter', 'g__Asteroleplasma']
                                               Feature    Importance
284  d__Bacteria;p__Firmicutes;c__Clostridia;o__Osc...  3.254419e-01
138  d__Bacteria;p__Desulfobacterota;c__Desulfovibr...  1.351138e-01
345  d__Bacteria;p__Firmicutes;c__Clostridia;o__Pep...  1.144176e-01
306  d__Bacteria;p__Firmicutes;c__Clostridia;o__Osc...  6.346977e-02
467  d__Bacteria;p__Spirochaetota;c__Spirochaetia;o...  4.132174e-02
174  d__Bacteria;p__Firmicutes;c__Bacilli;o__Izemop...  2.436586e-02
370  d__Bacteria;p__Firmicutes;c__Negativicutes;o__...  2.204238e-02
95   d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...  1.722158e-02
339  d__Bacteria;p__Firmicutes;c__Clostridia;o__Pep...  1.281327e-02
155  d__Bacteria;p__Firmicutes;c__Bacilli;o__Erysip...  1.280

In [ ]:
n_estimators = 500
random_seed = 434
X = raw_data[microbe_columns]
y = raw_data["Hexanoic Acid"]
rf = RandomForestRegressor(n_estimators=n_estimators, random_state=random_seed)
rf.fit(X, y)

importances = rf.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': microbe_columns, 'Importance': importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
top_features = feature_importance_df['Feature'][:10].values

print([i.split(';')[-1] for i in top_features])
print(feature_importance_df)